In [1]:
from collections import defaultdict, Counter
from intervaltree import Interval, IntervalTree
import itertools
import functools
import math
import numpy as np
import re
from typing import Callable, TypeVar

T = TypeVar('T')

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

In [99]:
data1 = data(1)

def day1(match):
    sum = 0
    for line in data1:
        matched = match(line)
        first, last = matched[0], matched[-1]
        # print(first, last, sum)
        sum += int(first + last)
    return sum

print(day1(lambda x: re.findall(r'\d', x)))

def part2match(x):
    change = {
        'one': '1',
        'two': '2',
        'three': '3',
        'four': '4',
        'five': '5',
        'six': '6',
        'seven': '7',
        'eight': '8',
        'nine': '9',
    }
    matched = re.findall(r'(?=(\d|one|two|three|four|five|six|seven|eight|nine))', x)
    return [change[match] if match in change else match for match in matched]

day1(part2match)
    

55816


54980

In [148]:
def format2(line):
    def tuplefy(round):
        result = [0,0,0]
        for i in round:
            result[0 if 'blue' in i else 1 if 'green' in i else 2] += int(i.split(' ')[0])
        return result

    line = line.split(': ')[-1]
    rounds = line.split('; ' )
    return np.array([tuplefy(x.split(', ')) for x in rounds])

data2 = data(2, format2)

def day2_1(start):
    result = 0
    for i, game in enumerate(data2):
        possible = np.subtract(start, game)
        if np.any(possible < 0):
            continue
        result += i+1
    return result

print(day2_1((14, 13, 12)))

def day2_2():
    result = 0
    for game in data2:
        requirements = np.amax(game, axis=0)
        result += functools.reduce(lambda x, y: x*y, requirements, 1)
    return result
day2_2()

2377


71220

In [130]:
data3 = data(3)

def day3():
    numbers = []
    for i, line in enumerate(data3):
        numbers += [(m.group(), (i, m.start())) for m in re.finditer(r'\d+', line)]

    array3 = np.pad(np.array([list(line) for line in data3]), ((1, 1), (1, 1)), constant_values='.')
    gears = defaultdict(lambda: [])

    part1 = 0
    for n, (i, j) in numbers:
        sliced = array3[i:i+3, j:j+2+len(n)]
        stringified =  ''.join(''.join(x) for x in sliced)
        if re.search(r'[^\d.]', stringified):
            part1 += int(n)
        if stars := np.where(sliced == '*'):
            for x, _ in enumerate(stars[0]):
                gears[(i+stars[0][x], j+stars[1][x])].append(int(n))

    part2 = 0
    for g in gears:
        if len(gears[g]) == 2:
            part2 += gears[g][0] * gears[g][1]

    return part1, part2

day3()

(521515, 69527306)

In [5]:
def parse4(line):
    winning, have = map(
        lambda numbers: {int(x) for x in numbers.split()},
        line.split(':')[-1].split('|')
    )
    return winning, have

data4 = data(4, parse4)

def day4(data):
    result, cards = 0, Counter()
    for i, line in enumerate(data):
        matches = line[0].intersection(line[1])
        if matches:
            result += 2**(len(matches)-1)
        for n in range(len(matches)):
            cards[i+1+n] += cards[i]+1
    return result, cards.total() + len(data)

day4(data4)


(25651, 19499881)

In [47]:
def day5_maps(data):
    names, maps = [], []

    def create_map(lines):
        ranges = IntervalTree([Interval(0, math.inf)])
        name = lines[0].split()[0]
        for i, line in enumerate(lines[1:]):
            if not line:
                break
            dest, source, size = map(int, line.split())
            ranges.chop(source, source+size)
            ranges[source:source+size] = dest
        return name, ranges, lines[i+2:]

    seeds, data = map(int, data[0].split(': ')[-1].split()), data[2:]

    while len(data):
        name, intervals, data = create_map(data)
        names.append(name)
        maps.append(intervals)

    return list(seeds), maps, names

seeds, intervals, names = day5_maps(data(5))

def day5_1(seeds, data):
    lowest = math.inf
    for seed in seeds:
        for interval in data:
            start, end, dest = next(iter(interval.at(seed)))
            seed = seed if dest is None else (seed-start)+dest
        lowest = min(lowest, seed)
    return lowest

def day5_2(seeds, data):
    seedrange = IntervalTree([Interval(start, start+size) for start, size in zip(seeds[::2], seeds[1::2])])
    seedrange.merge_overlaps()

    for i, interval in enumerate(data):
        newrange = IntervalTree()
        # print(names[i], seedrange, interval)
        for (seedstart, seedend, _) in seedrange:
            intersections = interval[seedstart:seedend]
            for (start, end, dest) in intersections:
                newstart = (dest + max(seedstart, start)-start) if dest is not None else seedstart
                newend = (dest + min(seedend, end)-start) if dest is not None else seedend
                newrange.addi(newstart, newend)
        newrange.merge_overlaps()
        seedrange = newrange
        
    lowest = min([loc[0] for loc in seedrange])
    return lowest

day5_1(seeds, intervals), day5_2(seeds, intervals)


(1181555926, 37806486)

In [38]:
data6 = data(6, lambda x: list(map(int, x.split(':')[-1].split())))

def day6(times, distances):
    result = []
    for t, d in zip(times, distances):
        # (t-x)*x = d+1
        # -x**2 +xt -d-1 = 0
        a, b, c = -1, t, -d-1
        det = math.sqrt(b**2 - 4*a*c)
        if det < 0:
            continue
        zeros = (-b+det)/(2*a), (-b-det)/(2*a)
        floor, ceil = math.ceil(min(zeros)), math.floor(max(zeros))
        result.append(ceil-floor+1)
    return functools.reduce(lambda x, y: x*y, result, 1)

day6(*data6), day6(*[[int(''.join(map(str, x)))] for x in data6])

(252000, 36992486)

In [34]:
data7 = data(7, lambda x: x.split())

In [48]:
def card_ord(card):
    match card:
        case 'T': return 10
        case 'J': return 11
        case 'Q': return 12
        case 'K': return 13
        case 'A': return 14
    return int(card)

def score_hand(hand):
    cards = [card_ord(card) for card in hand]
    counts = Counter(cards)
    score = sum([counts[x]**2 for x in counts])
    print(hand, score, counts)
    return [score]+cards

def day7(data):
    ordered = sorted(data, key=lambda x: score_hand(x[0]))
    return sum((i+1)*int(score) for i, (_, score) in enumerate(ordered))

day7(data7)

32T3K 7 Counter({3: 2, 2: 1, 10: 1, 13: 1})
T55J5 11 Counter({5: 3, 10: 1, 11: 1})
KK677 9 Counter({13: 2, 7: 2, 6: 1})
KTJJT 9 Counter({10: 2, 11: 2, 13: 1})
QQQJA 11 Counter({12: 3, 11: 1, 14: 1})


6440

In [35]:
data7

[['32T3K', '765'],
 ['T55J5', '684'],
 ['KK677', '28'],
 ['KTJJT', '220'],
 ['QQQJA', '483']]